# Keras Tuner Bars

In [1]:
import numpy as np
import pandas as pd
import time
import sys

sys.path.append("..")
from helpers import filename_for

Welcome to JupyROOT 6.16/00


In [2]:
import tensorflow
from tensorflow import keras
import kerastuner

print("tensorflow", tensorflow.__version__)
print("keras", keras.__version__)
print("keras-tuner", kerastuner.__version__)

tensorflow 2.1.0
keras 2.2.4-tf
keras-tuner 1.0.1


2020-07-01 09:51:35.359279: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/jmayer/Simulations/R3BRoot/build/lib:/data/jmayer/Simulations/2019-09-04/ucesb/hbook:/data/jmayer/Simulations/2019-09-04/FairRoot/lib:/data/jmayer/Simulations/2019-09-04/FairSoft/lib/root:/data/jmayer/Simulations/2019-09-04/FairSoft/lib
2020-07-01 09:51:35.359501: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/jmayer/Simulations/R3BRoot/build/lib:/data/jmayer/Simulations/2019-09-04/ucesb/hbook:/data/jmayer/Simulations/2019-09-04/FairRoot/lib:/data/jmayer/Simulations/2019-09-04/FairSoft/lib/root:/data/jmayer/Simulations/2019-09-04/FairSoft/lib
2020-07-01 0

In [3]:
features = ["nHits", "nClus", "Edep"] + [str(i) for i in range(0, 30 * 100 * 2)]
label = "nPN"
max_neutrons = 4
num_classes = max_neutrons + 1

In [4]:
start = time.time()
files = [
    filename_for(15, 30, 600, 500, n, "inclxx", s, "bars.parquet")
    for n in range(1, max_neutrons + 1)
    for s in range(20)
]
dfs = [pd.read_parquet(file) for file in files]
data = pd.concat(dfs, ignore_index=True).sample(frac=1)
print(time.time() - start)
display(data)

92.96923875808716


,nPN,nPP,nPH,nHits,nClus,Edep,0,1,2,3,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
541897,3,3,3,48,13,939,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
786279,4,3,3,41,14,802,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
587438,3,3,2,45,10,786,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
117831,1,0,0,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
680542,4,3,3,70,22,1041,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703763,4,4,4,80,18,1357,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27251,1,0,0,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30059,1,1,1,7,3,177,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39497,1,1,1,10,2,295,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
ys = keras.utils.to_categorical(data[[label]].values.ravel())#, num_classes=(max_neutrons))
Xs = data[features].values
#dataset = tf.data.Dataset.from_tensor_slices((, ys))

In [6]:
class Model(kerastuner.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.models.Sequential()
        for i in range(hp.Int("num_layers", 1, 2)):
            model.add(
                keras.layers.Dense(
                    units=hp.Int(f"units_{i}", min_value=1000, max_value=10000, step=3000), activation="relu",
                )
            )
        model.add(
            keras.layers.Dense(units=hp.Int(f"units_last", min_value=50, max_value=550, step=100), activation="relu",)
        )
        model.add(keras.layers.Dense(units=self.num_classes, activation="softmax"))

        optimizer = hp.Choice("optimizer", ["adam", "adagrad", "adadelta", "SGD"])
        loss = keras.losses.CategoricalCrossentropy()

        model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])

        return model

In [7]:
model = Model(num_classes)

# tuner = kerastuner.tuners.BayesianOptimization(
tuner = kerastuner.tuners.RandomSearch(
    model, objective="val_accuracy", max_trials=20, executions_per_trial=1, directory="kerastuner", project_name="V1",
)
tuner.search_space_summary()

In [8]:
tuner.search(
    Xs, ys, verbose=2, validation_split=0.1
)

Train on 720000 samples, validate on 80000 samples
720000/720000 - 992s - loss: 361279.8914 - accuracy: 0.5502 - val_loss: 10454.6479 - val_accuracy: 0.6068


Train on 720000 samples, validate on 80000 samples
720000/720000 - 1619s - loss: 147697.6737 - accuracy: 0.4033 - val_loss: 35194.6626 - val_accuracy: 0.4664


Train on 720000 samples, validate on 80000 samples
720000/720000 - 345s - loss: 199785.3276 - accuracy: 0.5562 - val_loss: 109159.2130 - val_accuracy: 0.5494


Train on 720000 samples, validate on 80000 samples
720000/720000 - 244s - loss: 359890.1101 - accuracy: 0.3246 - val_loss: 164315.1236 - val_accuracy: 0.3673


Train on 720000 samples, validate on 80000 samples
720000/720000 - 978s - loss: 386144.3598 - accuracy: 0.3518 - val_loss: 84940.6881 - val_accuracy: 0.3971


...

Train on 720000 samples, validate on 80000 samples
720000/720000 - 593s - loss: nan - accuracy: 1.4861e-04 - val_loss: nan - val_accuracy: 0.0000e+00


/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Train on 720000 samples, validate on 80000 samples
720000/720000 - 1867s - loss: 225258.9419 - accuracy: 0.4000 - val_loss: 62742.4680 - val_accuracy: 0.4599


INFO:tensorflow:Oracle triggered exit


2020-07-01 09:53:33.193079: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/jmayer/Simulations/R3BRoot/build/lib:/data/jmayer/Simulations/2019-09-04/ucesb/hbook:/data/jmayer/Simulations/2019-09-04/FairRoot/lib:/data/jmayer/Simulations/2019-09-04/FairSoft/lib/root:/data/jmayer/Simulations/2019-09-04/FairSoft/lib
2020-07-01 09:53:33.193286: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: UNKNOWN ERROR (303)
2020-07-01 09:53:33.193379: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (thanatos.ikp.uni-koeln.de): /proc/driver/nvidia/version does not exist
2020-07-01 09:53:33.193831: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020

In [9]:
tuner.results_summary()